In [1]:
from linear import *
from convolutional import *
from activations import *
from loss import CategoricalCrossEntropyLoss
import mnist

np.random.seed(0)

In [2]:
class MnistConv:
    def __init__(self, lr = 0.01) -> None:
        self.layers = [
            ConvLayer(1, 32, 5, activaton=ReLu()),
            MaxPool(2),
            Flatten(),
            Linear(32*12**2, 64, Sigmoid()),
            Linear(64, 10, None)
        ]
        self.lr = lr
        self.res = None
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        self.res = x
        return x
    
    def backward(self, expected_output):
        loss = CategoricalCrossEntropyLoss()
        loss_val = loss(self.res, expected_output)
        err = loss.derivatives()
        for layer in reversed(self.layers):
            err = layer.backward(err, self.lr)
        return loss_val

In [3]:
x_train, t_train, x_test, t_test = mnist.load()
model = MnistConv()

In [4]:
EPOCHES = 3

for epoch in range(EPOCHES):
    print(f"epoch {epoch}/{EPOCHES}")
    loss_val = 0
    for i, (img, label) in enumerate(zip(x_train[:10000], t_train[:10000])):

        res = model.forward(img.reshape((1, 1, 28, 28))/255.0)
        loss_val += model.backward(label)
        if i % 100 == 0 and i > 0:
            print(f"epoch: {epoch}.{i//100}, loss: {loss_val/100}")
            loss_val = 0




epoch 0/3
epoch: 0.1.0, loss: 3.199688172315947
epoch: 0.2.0, loss: 2.2314858475308976
epoch: 0.3.0, loss: 2.021405788811015
epoch: 0.4.0, loss: 1.5823702095279293
epoch: 0.5.0, loss: 1.8816727679706833
epoch: 0.6.0, loss: 2.1217259969299236
epoch: 0.7.0, loss: 1.929911914713228
epoch: 0.8.0, loss: 1.6655691875115846
epoch: 0.9.0, loss: 1.8361567768318605
epoch: 0.10.0, loss: 1.8168993566179426
epoch: 0.11.0, loss: 1.8148235418658456
epoch: 0.12.0, loss: 1.528723670805916
epoch: 0.13.0, loss: 1.5329643781380033
epoch: 0.14.0, loss: 1.4491535657778383
epoch: 0.15.0, loss: 1.406615574401152
epoch: 0.16.0, loss: 1.5046553666662836
epoch: 0.17.0, loss: 1.2025224937734946
epoch: 0.18.0, loss: 1.2096568604376552
epoch: 0.19.0, loss: 1.1817918193327244
epoch: 0.20.0, loss: 1.38739562114119
epoch: 0.21.0, loss: 1.2520921024059386
epoch: 0.22.0, loss: 1.0916106636094969
epoch: 0.23.0, loss: 1.2802884430673243
epoch: 0.24.0, loss: 1.1094613274095828
epoch: 0.25.0, loss: 1.1986670306913965
epoch:

In [8]:
results = np.zeros((3, 10))
for (img, label) in zip(x_test[:1000], t_test[:1000]):
    results[0][label] += 1
    r = model.forward(img.reshape((1, 1, 28, 28))/255.0)
    if np.argmax(r, axis = 1) == label:
        results[1][label] +=1
    results[2][np.argmax(r, axis = 1)] += 1

print(results[1])
print(results[0])
print(results[2])

[ 81. 121.  95.  86.  94.  68.  72.  78.  64.  70.]
[ 85. 126. 116. 107. 110.  87.  87.  99.  89.  94.]
[100. 129. 114.  97. 113.  87.  83.  89. 102.  86.]


In [9]:
np.sum(results[1])/np.sum(results[0])

0.829

In [7]:
results[1]/results[0]

array([0.97938144, 0.93103448, 0.76767677, 0.76344086, 0.91428571,
       0.63043478, 0.88297872, 0.83760684, 0.8045977 , 0.7       ])